# Real time Digits!

In [ ]:
### Main header.

import numpy as np
import cv2
import torch
import torch.nn as nn
import torch.nn.functional as F

from torch.autograd import Variable
from torchvision import transforms

In [ ]:
### Network class definition and deserialization.

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(in_channels  = 1,
                               out_channels = 10,
                               kernel_size  = 5)
        
        self.conv2 = nn.Conv2d(in_channels  = 10,
                               out_channels = 20,
                               kernel_size  = 5)
        
        self.conv2_drop = nn.Dropout2d()
        
        self.fc1 = nn.Linear(in_features  = 320,
                             out_features = 50)
        
        self.fc2 = nn.Linear(in_features  = 50,
                             out_features = 10)
        
    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

model = Net()
model.load_state_dict(torch.load("./state.pkl"))

In [ ]:
def classify_digit(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)  # To gray
    image = cv2.resize(image, (28, 28))              # Resize
    
    image  # 
    image = np.array(image, dtype=np.uint8)                             #

    image = cv2.bitwise_not(image)                   # Invert

    f = transforms.ToTensor()                       # ToTensor() converts numpy.ndarray (H x W x C) in the range
    image = np.reshape(image, (28,28,1))            # [0, 255] to a torch.FloatTensor of shape (C x H x W)
    image = f(image)                                # in the range [0.0, 1.0].

    array = np.array(image)                            # Normalization
    f = transforms.Normalize((float(np.mean(array)),), #
                             (float(np.std(array)),))  #
    #f = transforms.Normalize((0.1307,),(0.3081,))
    image = f(image)                                   #
    
    image = image.unsqueeze(0) # Adds the groups dimension. Tensor shape now is (1,1,28,28).
    
    model.eval()
    image = Variable(image)
    result = model(image)
    
    return np.argsort(np.array(result.data[0]))[-1]

In [ ]:
### Real time classification!!

cap = cv2.VideoCapture(0)

while(True):
    ret, frame = cap.read()
    
    number = str(classify_digit(frame, model))
    
    font = cv2.FONT_HERSHEY_SIMPLEX
    cv2.putText(frame,number,(10,100), font, 4,(255,255,255),2,cv2.LINE_AA)
    
    cv2.imshow('frame', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()